<a href="https://colab.research.google.com/github/krishds20/RNN/blob/main/RNN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,StandardScaler,LabelEncoder
from sklearn.metrics import accuracy_score,precision_score,f1_score,recall_score

In [ ]:
df=pd.read_csv('/content/sample_data/drug adverse reaction.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7386 entries, 0 to 7385
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   age                   7386 non-null   int64  
 1   sex                   7386 non-null   object 
 2   height_cm             7386 non-null   float64
 3   weight_kg             7386 non-null   float64
 4   BMI                   7386 non-null   float64
 5   medical_history       5927 non-null   object 
 6   drug_name             7386 non-null   object 
 7   condition             7386 non-null   object 
 8   dosage_mg             7386 non-null   int64  
 9   side_effect           6332 non-null   object 
 10  side_effect_severity  6332 non-null   object 
dtypes: float64(3), int64(2), object(6)
memory usage: 634.9+ KB


In [ ]:
df['side_effect']=df['side_effect'].fillna('No Side Efffect')

In [ ]:
features=['age','sex','height_cm','weight_kg','BMI','medical_history','drug_name','dosage_mg']
x=df[features]
y=df['side_effect']

In [ ]:
le=LabelEncoder()
y_encoded=le.fit_transform(y)

In [ ]:
num_features=['age','height_cm','weight_kg','BMI','dosage_mg']
cat_features=['sex','medical_history','drug_name']

In [ ]:
preprocessor=ColumnTransformer(
    transformers=[
        ('num',StandardScaler(),num_features),
        ('cat',OneHotEncoder(handle_unknown='ignore'),cat_features)
    ]
)

In [ ]:
X_transformed=preprocessor.fit_transform(x)


In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X_transformed,y_encoded,test_size=0.2,random_state=42)

In [ ]:
if hasattr(x_train,'toarray'):
  x_train_nn=x_train.toarray()
  x_test_nn=x_test.toarray()
else:
  x_train_nn=x_train
  x_test_nn=x_test

In [ ]:
x_train_nn=np.array(x_train_nn)
x_test_nn=np.array(x_test_nn)
x_train_nn=x_train_nn.reshape(x_train_nn.shape[0],x_train_nn.shape[1],1)
x_test_nn=x_test_nn.reshape(x_test_nn.shape[0],x_test_nn.shape[1],1)
input_shape=(x_train_nn.shape[1],1)

In [ ]:
import tensorflow as tf
num_classes=len(le.classes_)
y_train_cat=tf.keras.utils.to_categorical(y_train,num_classes)
y_test_cat=tf.keras.utils.to_categorical(y_test,num_classes)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D,Flatten,Dense
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
from tensorflow.keras.layers import LSTM

rnn_model = Sequential([
    LSTM(32, input_shape=input_shape),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
rnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print("\nTraining RNN model for multi-class side effect prediction...")
rnn_model.fit(x_train_nn, y_train_cat, epochs=15, batch_size=32,
              validation_split=0.2, verbose=1,
              callbacks=[EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)])


Training RNN model for multi-class side effect prediction...
Epoch 1/15
148/148 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.1396 - loss: 3.5876 - val_accuracy: 0.1396 - val_loss: 3.2726
Epoch 2/15
148/148 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.1379 - loss: 3.2752 - val_accuracy: 0.1396 - val_loss: 3.2715
Epoch 3/15
148/148 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - accuracy: 0.1404 - loss: 3.2793 - val_accuracy: 0.1396 - val_loss: 3.2790
Epoch 4/15
148/148 ━━━━━━━━━━━━━━━━━━━━ 6s 38ms/step - accuracy: 0.1454 - loss: 3.2674 - val_accuracy: 0.1396 - val_loss: 3.2725
Epoch 5/15
148/148 ━━━━━━━━━━━━━━━━━━━━ 6s 38ms/step - accuracy: 0.1427 - loss: 3.2581 - val_accuracy: 0.1396 - val_loss: 3.2612
Epoch 6/15
148/148 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - accuracy: 0.1426 - loss: 3.2542 - val_accuracy: 0.1396 - val_loss: 3.2186
Epoch 7/15
148/148 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.1436 - loss: 3.1447 - val_accuracy: 0.1387 - val_loss: 2.8501
Epoch 8/15
148/148 ━━━━━━━━━━━━━━━━

In [ ]:
rnn_loss, rnn_accuracy = rnn_model.evaluate(x_test_nn, y_test_cat, verbose=0)
print(f"\nRNN Model Accuracy: {rnn_accuracy:.4f}")


RNN Model Accuracy: 0.2226


In [ ]:
y_pred_rnn = rnn_model.predict(x_test_nn)
y_pred_rnn_labels = np.argmax(y_pred_rnn, axis=1)
print("\nRNN Model Metrics:")
print("Accuracy:", accuracy_score(y_test, y_pred_rnn_labels))
print("Precision:", precision_score(y_test, y_pred_rnn_labels, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_rnn_labels, average='weighted'))
print("F1 Score:", f1_score(y_test, y_pred_rnn_labels, average='weighted'))

In [ ]:
#making prediction
new_data = {
    'age': [30],
    'sex': ['Female'],
    'height_cm': [160],
    'weight_kg': [55],
    'BMI': [21.5],
    'medical_history': ['Diabetes'],
    'drug_name': ['Zolpidem'],
    'condition': ['Insomnia'],
    'dosage_mg': [100],
}

In [ ]:
new_df = pd.DataFrame(new_data)
new_data_transformed = preprocessor.transform(new_df)

In [ ]:
if hasattr(new_data_transformed, "toarray"):
    new_data_transformed = new_data_transformed.toarray()
new_data_nn = np.array(new_data_transformed)
# Reshape into (samples, timesteps, channels)
new_data_nn = new_data_nn.reshape(new_data_nn.shape[0], new_data_nn.shape[1], 1)

In [ ]:
rnn_pred_prob = rnn_model.predict(new_data_nn)
rnn_pred_index = np.argmax(rnn_pred_prob, axis=1)
rnn_pred_label = le.inverse_transform(rnn_pred_index)
print("RNN Prediction:", rnn_pred_label[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 414ms/step
RNN Prediction: dizziness
